In [1]:
# ================================
# Standard library
# ================================
import os
import re
from datetime import datetime
from pprint import pprint

# ================================
# Scientific computing
# ================================
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.fft import fft, fftfreq
from scipy.signal import hilbert, welch
from scipy.sparse import coo_matrix

# ================================
# Machine learning - core
# ================================
import lightgbm as lgb
from xgboost import XGBRegressor

# ================================
# Machine learning - scikit-learn
# ================================
import joblib
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.metrics import (
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_val_score,
    train_test_split,
)
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# ================================
# Progress bar
# ================================
from tqdm.auto import tqdm
from tqdm_joblib import tqdm_joblib


In [ ]:
# Converts relative paths to absolute ones
ROOT_TRAIN_DATA_FOLDER ='../data/raw/B - PHM America 2023 - Dataset/Data_Challenge_PHM2023_training_data'
ROOT_TEST_DATA_FOLDER = '../data/raw/B - PHM America 2023 - Dataset/Data_Challenge_PHM2023_test_data'

## Parsing

In [ ]:
def parse_vibration_dataset(dataset_path):
    """
    Parsa il dataset di vibrazione e crea un DataFrame pandas con una riga per ogni file.
    Ogni riga contiene gli array completi delle serie temporali di vibrazione.

    Args:
        dataset_path (str): Percorso alla cartella principale del dataset

    Returns:
        pd.DataFrame: DataFrame con colonne etichetta, velocità, torque, rep e array di dati di vibrazione (una riga per file)
    """

    data_list = []
    
    # Prima passata: conta il numero totale di file .txt
    print("Conteggio file in corso...")
    total_files = 0
    for root, dirs, files in os.walk(dataset_path):
        total_files += sum(1 for file in files if file.endswith('.txt'))
    
    print(f"Trovati {total_files} file .txt da processare")

    # Seconda passada: processa i file con progress bar
    with tqdm(total=total_files, desc="Parsing dataset", unit="file") as pbar:
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.endswith('.txt'):
                    folder_name = os.path.basename(root)
                    if 'Pitting_degradation_level_' in folder_name:
                        etichetta_full = folder_name.replace('Pitting_degradation_level_', '')
                        if '(' in etichetta_full:
                            etichetta = etichetta_full.split('(')[0].strip()
                            descrizione = etichetta_full.split('(')[1].replace(')', '').strip()
                        else:
                            etichetta = etichetta_full.strip()
                            descrizione = None
                    else:
                        etichetta = folder_name
                        descrizione = None

                    # ERRORE CORRETTO: regex pattern per V100_200N_2.txt
                    pattern = r'V(\d+)_(\d+)N_(\d+)\.txt'
                    match = re.search(pattern, file)

                    if match:
                        velocita = int(match.group(1))
                        torque = int(match.group(2))
                        rep = int(match.group(3))

                        file_path = os.path.join(root, file)
                        try:
                            # Carica tutti i dati del file una volta sola
                            data = np.loadtxt(file_path)
                            
                            # Calcola informazioni aggiuntive sui dati
                            sampling_rate = 20480  # Hz come specificato nella documentazione
                            duration = len(data) / sampling_rate
                            
                            # Crea un record per l'intero file
                            record = {
                                'file_name': file,
                                'etichetta': etichetta,
                                'health_level': int(etichetta) if etichetta.isdigit() else etichetta,
                                'velocita': velocita,
                                'torque': torque,
                                'rep': rep,
                                'horizontal_acceleration': data[:, 0],  # Array completo
                                'axial_acceleration': data[:, 1],       # Array completo
                                'vertical_acceleration': data[:, 2],    # Array completo
                                'tachometer_signal': data[:, 3],        # Array completo
                                'sampling_rate': sampling_rate,
                                'duration': duration,
                                'num_samples': len(data),
                                'descrizione': descrizione
                            }
                            # ERRORE CORRETTO: aggiungi record invece di data
                            data_list.append(record)

                        except Exception as e:
                            tqdm.write(f"Errore nel leggere il file {file_path}: {e}")
                            continue
                    else:
                        tqdm.write(f"Nome file non riconosciuto: {file}")
                    
                    # Aggiorna la progress bar
                    pbar.update(1)

    df = pd.DataFrame(data_list)

    if not df.empty:
        print("\nOrdinamento dataset...")
        df = df.sort_values(['health_level', 'velocita', 'torque', 'rep']).reset_index(drop=True)
        print(f"Dataset caricato: {len(df)} file processati")
        print(f"Health levels disponibili: {sorted(df['health_level'].unique())}")
        print(f"Condizioni operative (rpm): {sorted(df['velocita'].unique())}")
        print(f"Condizioni operative (torque): {sorted(df['torque'].unique())}")

    df.to_pickle('../data/processed/train_data.pkl')
    
    return df

# Esempio di utilizzo:
# Carica il dataset
print("=== CARICAMENTO DATASET ===")
df = parse_vibration_dataset(ROOT_TRAIN_DATA_FOLDER)

df.head()

In [ ]:
def parse_test_dataset(dataset_path):
    """
    Parsa il dataset di vibrazione e crea un DataFrame pandas con una riga per ogni file.
    Ogni riga contiene gli array completi delle serie temporali di vibrazione.

    Args:
        dataset_path (str): Percorso alla cartella principale del dataset

    Returns:
        pd.DataFrame: DataFrame con colonne etichetta, velocità, torque, rep e array di dati di vibrazione (una riga per file)
    """

    data_list = []
    
    # Prima passata: conta il numero totale di file .txt
    print("Conteggio file in corso...")
    total_files = 0
    for root, dirs, files in os.walk(dataset_path):
        total_files += sum(1 for file in files if file.endswith('.txt'))
    
    print(f"Trovati {total_files} file .txt da processare")

    # Seconda passada: processa i file con progress bar
    with tqdm(total=total_files, desc="Parsing dataset", unit="file") as pbar:
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.endswith('.txt'):

                    # ERRORE CORRETTO: regex pattern per V100_200N_2.txt
                    pattern = re.compile(r'^(\d+)_V(\d+)_(\d+)N(?:_(\d+))?\.txt$', re.IGNORECASE)
                    match = re.search(pattern, file)

                    if match:
                        velocita = int(match.group(2))
                        torque = int(match.group(3))
                        id = int(match.group(1))

                        file_path = os.path.join(root, file)
                        try:
                            # Carica tutti i dati del file una volta sola
                            data = np.loadtxt(file_path)
                            
                            # Calcola informazioni aggiuntive sui dati
                            sampling_rate = 20480  # Hz come specificato nella documentazione
                            duration = len(data) / sampling_rate
                            
                            # Crea un record per l'intero file
                            record = {
                                'file_name': file,
                                'id': id,
                                'velocita': velocita,
                                'torque': torque,
                                'horizontal_acceleration': data[:, 0],  # Array completo
                                'axial_acceleration': data[:, 1],       # Array completo
                                'vertical_acceleration': data[:, 2],    # Array completo
                                'tachometer_signal': data[:, 3],        # Array completo
                                'sampling_rate': sampling_rate,
                                'duration': duration,
                                'num_samples': len(data),
                            }
                            # ERRORE CORRETTO: aggiungi record invece di data
                            data_list.append(record)

                        except Exception as e:
                            tqdm.write(f"Errore nel leggere il file {file_path}: {e}")
                            continue
                    else:
                        tqdm.write(f"Nome file non riconosciuto: {file}")
                    
                    # Aggiorna la progress bar
                    pbar.update(1)

    df = pd.DataFrame(data_list)

    if not df.empty:
        print("\nOrdinamento dataset...")
        df = df.sort_values(['velocita', 'torque']).reset_index(drop=True)
        print(f"Dataset caricato: {len(df)} file processati")
        print(f"Condizioni operative (rpm): {sorted(df['velocita'].unique())}")
        print(f"Condizioni operative (torque): {sorted(df['torque'].unique())}")

    df.to_pickle('../data/processed/test_data.pkl')

    return df


test_df = parse_test_dataset(ROOT_TEST_DATA_FOLDER)

## Downsampling

In [ ]:
df = pd.read_pickle('../data/processed/train_data.pkl')

In [ ]:
def downsample(signal, fs=20480, sec=3):
    """
    Filtra e fa il resample di un segnale ad una lunghezza fissa.
    """
    if len(signal) == fs*sec:
        return signal
    return signal[:fs*sec]  # Truncate or pad to the desired length

def downsample_vibration_dataframe(df, sec=3):
    """
    Applica filtraggio e resample a tutti i segnali accelerometrici e tachimetro nel DataFrame.
    Ritorna un nuovo DataFrame con colonne preprocessate.
    """
    processed_records = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Preprocessing"):
        try:
            record = {
                'file_name': row['file_name'],
                'etichetta': row['etichetta'],
                'health_level': row['health_level'],
                'velocita': row['velocita'],
                'torque': row['torque'],
                'rep': row['rep'],
                'sampling_rate': row['sampling_rate'],
                'descrizione': row['descrizione'],
                'duration': row['duration'],
                'num_samples': row['num_samples']
            }

            # Applica filtro + resample ai 4 segnali
            record['horizontal_acceleration'] = downsample(
                row['horizontal_acceleration'], fs=row['sampling_rate'], sec=sec
            )
            record['axial_acceleration'] = downsample(
                row['axial_acceleration'], fs=row['sampling_rate'], sec=sec
            )
            record['vertical_acceleration'] = downsample(
                row['vertical_acceleration'], fs=row['sampling_rate'], sec=sec
            )
            record['tachometer_signal'] = downsample(
                row['tachometer_signal'], fs=row['sampling_rate'], sec=sec
            )

            processed_records.append(record)

        except Exception as e:
            tqdm.write(f"Errore durante il preprocessing del file {row['file_name']}: {e}")
            continue

    return pd.DataFrame(processed_records)


In [ ]:
features_df = downsample_vibration_dataframe(df, sec=3)

In [ ]:
features_df.to_pickle('../data/processed/train_data_dowsampled_walt.pkl')

In [2]:
features_df = pd.read_pickle("../data/processed/train_data_dowsampled_walt.pkl")

## Feature extraction

In [3]:
# -----------------------
# 1) Pulizia base target
# -----------------------
features_df['health_level'] = pd.to_numeric(features_df['health_level'], errors='coerce')
features_df = features_df.dropna(subset=['health_level']).reset_index(drop=True)

# -----------------------
# 2) Definisci colonne
# -----------------------
scalar_cols = ['velocita', 'torque']  # numeriche scalari già pronte
array_cols = ['horizontal_acceleration', 'axial_acceleration', 'vertical_acceleration', 'tachometer_signal']

scalar_cols = [c for c in scalar_cols if c in features_df.columns]
array_cols  = [c for c in array_cols  if c in features_df.columns]

# Parametri segnale
FS = 20480  # Hz (dal task PHM)
PULSES_PER_REV = 1
TACH_COL_NAME = 'tachometer_signal'

# -----------------------
# 3) Utils
# -----------------------
def _safe_array(x):
    if isinstance(x, (list, tuple, np.ndarray, pd.Series)):
        arr = np.asarray(x, dtype=float).ravel()
    else:
        try:
            arr = np.asarray([x], dtype=float)
        except Exception:
            arr = np.asarray([], dtype=float)
    arr = arr[np.isfinite(arr)]
    return arr

def time_feats(arr):
    if arr.size == 0:
        return {
            'mean': np.nan, 'std': np.nan, 'min': np.nan, 'max': np.nan,
            'median': np.nan, 'p10': np.nan, 'p90': np.nan, 'rms': np.nan,
            'skew': np.nan, 'kurt': np.nan, 'ptp': np.nan, 'iqr': np.nan,
            'zcr': np.nan, 'envelope_rms': np.nan
        }
    feats = {
        'mean': float(np.mean(arr)),
        'std': float(np.std(arr)),
        'min': float(np.min(arr)),
        'max': float(np.max(arr)),
        'median': float(np.median(arr)),
        'p10': float(np.percentile(arr, 10)),
        'p90': float(np.percentile(arr, 90)),
        'rms': float(np.sqrt(np.mean(arr**2))),
        'skew': float(stats.skew(arr, bias=False)) if arr.size > 2 else 0.0,
        'kurt': float(stats.kurtosis(arr, fisher=True, bias=False)) if arr.size > 3 else 0.0,
        'ptp': float(np.ptp(arr)),
        'iqr': float(np.subtract(*np.percentile(arr, [75, 25]))),
        'zcr': float(np.mean(np.abs(np.diff(np.sign(arr))) > 0)) if arr.size > 1 else 0.0,
    }
    try:
        env = np.abs(hilbert(arr))
        feats['envelope_rms'] = float(np.sqrt(np.mean(env**2)))
    except Exception:
        feats['envelope_rms'] = np.nan
    return feats

def tach_feats(arr, fs=FS, pulses_per_rev=PULSES_PER_REV, threshold=0.0):
    if arr.size == 0:
        return {'rpm': np.nan, 'pulse_count': 0, 'ipi_mean': np.nan, 'ipi_std': np.nan, 'ipi_cv': np.nan}
    bin_sig = (arr > threshold).astype(np.uint8)
    idx = np.flatnonzero(bin_sig)
    pulse_count = int(idx.size)
    rpm = np.nan
    if fs is not None and arr.size > 0 and pulses_per_rev > 0:
        window_sec = arr.size / float(fs)
        if window_sec > 0:
            revs = pulse_count / float(pulses_per_rev)
            rpm = (revs / window_sec) * 60.0
    if idx.size > 1:
        ipi = np.diff(idx) / float(fs) if fs else np.diff(idx).astype(float)
        ipi_mean = float(np.mean(ipi))
        ipi_std  = float(np.std(ipi))
        ipi_cv   = float(ipi_std / ipi_mean) if ipi_mean > 0 else np.nan
    else:
        ipi_mean = ipi_std = ipi_cv = np.nan
    return {'rpm': float(rpm), 'pulse_count': pulse_count, 'ipi_mean': ipi_mean, 'ipi_std': ipi_std, 'ipi_cv': ipi_cv}

def psd_band_feats(arr, fs=FS, bands=None):
    if bands is None:
        # bande fino alla Nyquist (10240 Hz), regolabili
        bands = [(0,200),(200,500),(500,1000),(1000,2000),(2000,4000),(4000,8000)]
    if arr.size < 8:
        return {f'band_{lo}_{hi}': np.nan for (lo,hi) in bands}
    # Welch PSD
    try:
        f, Pxx = welch(arr, fs=fs, nperseg=min(2048, len(arr)))
    except Exception:
        return {f'band_{lo}_{hi}': np.nan for (lo,hi) in bands}
    feats = {}
    for (lo, hi) in bands:
        mask = (f >= lo) & (f < hi)
        feats[f'band_{lo}_{hi}'] = float(np.trapz(Pxx[mask], f[mask])) if np.any(mask) else 0.0
    # normalizza per energia totale per robustezza
    total = float(np.trapz(Pxx, f)) if f.size else 1.0
    for (lo, hi) in bands:
        key = f'band_{lo}_{hi}'
        feats[key] = feats[key] / total if total > 0 else 0.0
    return feats

def cross_axis_feats(ax, ay, az):
    feats = {}
    def safe_corr(a,b):
        if len(a) < 2 or len(b) < 2:
            return np.nan
        a = a - np.mean(a)
        b = b - np.mean(b)
        denom = (np.std(a)*np.std(b))
        if denom == 0:
            return 0.0
        return float(np.mean(a*b)/denom)
    feats['corr_xy'] = safe_corr(ax, ay)
    feats['corr_xz'] = safe_corr(ax, az)
    feats['corr_yz'] = safe_corr(ay, az)
    return feats

# -----------------------
# 4) Espansione feature
# -----------------------
def expand_features(df, array_cols, scalar_cols):
    rows = []
    for i, row in df.iterrows():
        feat_row = {}
        # scalari
        for c in scalar_cols:
            feat_row[c] = row[c]
        # array
        ax = _safe_array(row[array_cols[0]]) if 'horizontal_acceleration' in array_cols else np.array([])
        ay = _safe_array(row[array_cols[1]]) if 'axial_acceleration' in array_cols else np.array([])
        az = _safe_array(row[array_cols[2]]) if 'vertical_acceleration' in array_cols else np.array([])
        tach = _safe_array(row[TACH_COL_NAME]) if TACH_COL_NAME in array_cols else np.array([])

        # time feats per-asse
        for sig, name in [(ax,'ax'), (ay,'ay'), (az,'az')]:
            if sig.size > 0:
                tf = time_feats(sig)
                tf = {f'{name}_{k}': v for k,v in tf.items()}
                feat_row.update(tf)

                pf = psd_band_feats(sig, fs=FS)
                pf = {f'{name}_{k}': v for k,v in pf.items()}
                feat_row.update(pf)

        # cross-axis
        if ax.size > 0 and ay.size > 0 and az.size > 0:
            feat_row.update(cross_axis_feats(ax, ay, az))

        # tach
        if tach.size > 0:
            tfeat = tach_feats(tach, fs=FS, pulses_per_rev=PULSES_PER_REV, threshold=0.0)
            feat_row.update(tfeat)

        # interazioni operative
        if 'velocita' in feat_row and 'torque' in feat_row:
            v = feat_row['velocita']
            t = feat_row['torque']
            feat_row['v_times_t'] = v * t
            feat_row['v_sq'] = v**2
            feat_row['t_sq'] = t**2
            if 'rpm' in feat_row and np.isfinite(feat_row['rpm']):
                rpm = feat_row['rpm']
                # normalizzazioni semplici
                feat_row['v_over_rpm'] = v / (rpm + 1e-6)
                feat_row['t_over_rpm'] = t / (rpm + 1e-6)

        rows.append(feat_row)

    X_full = pd.DataFrame(rows)
    # imputazione semplice
    X_full = X_full.replace([np.inf, -np.inf], np.nan)
    X_full = X_full.fillna(X_full.median(numeric_only=True))
    return X_full


In [4]:
print("[*] Genero feature...")
X_full = expand_features(features_df, array_cols=array_cols, scalar_cols=scalar_cols)
y = features_df['health_level'].astype(float).values

print(f"[*] Shape X_full: {X_full.shape}, n_features = {X_full.shape[1]}")

[*] Genero feature...
[*] Shape X_full: (2016, 75), n_features = 75


In [5]:
X_full

,velocita,torque,ax_mean,ax_std,ax_min,ax_max,ax_median,ax_p10,ax_p90,ax_rms,...,rpm,pulse_count,ipi_mean,ipi_std,ipi_cv,v_times_t,v_sq,t_sq,v_over_rpm,t_over_rpm
0,100,50,-0.169319,0.162043,-0.794698,0.118969,-0.136461,-0.413488,0.017492,0.234365,...,60.0,3,1.082007,0.000659,0.000609,5000,10000,2500,1.666667,0.833333
1,100,50,0.000265,0.071211,-0.282226,0.157426,0.003846,-0.097011,0.089568,0.071211,...,60.0,3,1.082153,0.000220,0.000203,5000,10000,2500,1.666667,0.833333
2,100,50,-0.000159,0.074463,-0.336066,0.162016,0.003598,-0.102221,0.093041,0.074463,...,40.0,2,1.081201,0.000000,0.000000,5000,10000,2500,2.500000,1.250000
3,100,50,-0.000540,0.075078,-0.290537,0.164125,0.003101,-0.103723,0.093662,0.075080,...,40.0,2,1.082666,0.000000,0.000000,5000,10000,2500,2.500000,1.250000
4,100,50,-0.000262,0.075467,-0.300958,0.184594,0.003722,-0.103090,0.094406,0.075467,...,60.0,3,1.082373,0.000195,0.000180,5000,10000,2500,1.666667,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,3600,50,0.003271,0.936696,-4.257450,3.650200,0.005645,-1.185720,1.187097,0.936702,...,2000.0,100,0.030001,0.000024,0.000802,180000,12960000,2500,1.800000,0.025000
2012,3600,50,-0.000045,0.932316,-4.302978,4.262537,0.001116,-1.186712,1.181998,0.932316,...,2000.0,100,0.030002,0.000024,0.000807,180000,12960000,2500,1.800000,0.025000
2013,3600,100,-0.001060,1.102834,-4.373690,4.765580,0.004094,-1.419564,1.389803,1.102835,...,2000.0,100,0.030002,0.000024,0.000807,360000,12960000,10000,1.800000,0.050000
2014,3600,100,-0.003609,1.074484,-4.859490,4.633213,0.003349,-1.377658,1.362250,1.074491,...,2000.0,100,0.030001,0.000024,0.000804,360000,12960000,10000,1.800000,0.050000


## Training

In [6]:
# -----------------------
# 5) Split
# -----------------------
y_int = y.astype(int)
X = X_full.astype(np.float32).values

X_tr, X_va, y_tr, y_va = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_int
)

In [7]:
# -----------------------
# 6) Model selection - Enhanced with XGBoost and additional algorithms
# -----------------------

# Random Forest (già presente)
rf = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
])

rf_param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__max_depth": [6, 8, 10, 12, None],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
}

# Histogram Gradient Boosting (già presente)
hgb = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", HistGradientBoostingRegressor(random_state=42))
])

hgb_param_dist = {
    "model__max_depth": [None, 4, 6, 8],
    "model__learning_rate": [0.03, 0.05, 0.1],
    "model__max_iter": [200, 400, 800],
    "model__l2_regularization": [0.0, 0.1, 1.0],
    "model__min_samples_leaf": [10, 20, 30, 50],
}

# XGBoost - NUOVO
xgb = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))
])

xgb_param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__max_depth": [3, 4, 6, 8],
    "model__learning_rate": [0.01, 0.05, 0.1, 0.15],
    "model__subsample": [0.8, 0.9, 1.0],
    "model__colsample_bytree": [0.8, 0.9, 1.0],
    "model__reg_alpha": [0, 0.1, 1],
    "model__reg_lambda": [1, 1.5, 2],
}

# LightGBM - NUOVO
lgbm = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbosity=-1))
])

lgbm_param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__max_depth": [3, 4, 6, 8, -1],
    "model__learning_rate": [0.01, 0.05, 0.1, 0.15],
    "model__subsample": [0.8, 0.9, 1.0],
    "model__colsample_bytree": [0.8, 0.9, 1.0],
    "model__reg_alpha": [0, 0.1, 1],
    "model__reg_lambda": [0, 0.1, 1],
    "model__num_leaves": [31, 50, 100, 200],
}

# Gradient Boosting Classico - NUOVO
gb = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", GradientBoostingRegressor(random_state=42))
])

gb_param_dist = {
    "model__n_estimators": [200, 400, 600],
    "model__max_depth": [3, 4, 6, 8],
    "model__learning_rate": [0.01, 0.05, 0.1, 0.15],
    "model__subsample": [0.8, 0.9, 1.0],
    "model__max_features": ["sqrt", "log2", 0.3, 0.5],
}

# Extra Trees - NUOVO
et = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("model", ExtraTreesRegressor(random_state=42, n_jobs=-1))
])

et_param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__max_depth": [6, 8, 10, 12, None],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__min_samples_split": [2, 5, 10],
    "model__max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
}

# Ridge Regression 
ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(random_state=42))
])

ridge_param_dist = {
    "model__alpha": [0.1, 1.0, 10.0, 100.0, 1000.0],
}

# Elastic Net
elastic = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ElasticNet(random_state=42))
])

elastic_param_dist = {
    "model__alpha": [0.1, 1.0, 10.0, 100.0],
    "model__l1_ratio": [0.1, 0.5, 0.7, 0.9],
}

def fit_search(name, pipe, param_dist, *, n_iter=25, cv=3):
    print(f"\n[*] Tuning {name}...")
    search = RandomizedSearchCV(
        pipe,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring="neg_mean_absolute_error",
        cv=cv,
        verbose=0,              # lascia a 0: la progress bar si occupa dell'output
        random_state=42,
        n_jobs=-1
    )

    # Numero totale di fit = n_iter * n_fold (circa; gli errori/skipped possono ridurre il totale)
    total_steps = n_iter * (cv if isinstance(cv, int) else cv.get_n_splits(X_tr, y_tr))

    with tqdm_joblib(tqdm(total=total_steps, desc=f"Tuning {name}", leave=True)):
        search.fit(X_tr, y_tr)

    print(f"[+] Best MAE (cv) {name}: {-search.best_score_:.4f}")
    print(f"[+] Best params {name}:")
    pprint(search.best_params_)
    return search

# Training di tutti i modelli
print("=" * 60)
print("TRAINING MODELLI")
print("=" * 60)

rf_search = fit_search("RandomForest", rf, rf_param_dist, n_iter=25, cv=3)
hgb_search = fit_search("HistGradientBoosting", hgb, hgb_param_dist, n_iter=25, cv=3)
xgb_search = fit_search("XGBoost", xgb, xgb_param_dist, n_iter=30, cv=3)  # Più iterazioni per XGB
lgbm_search = fit_search("LightGBM", lgbm, lgbm_param_dist, n_iter=30, cv=3)
gb_search = fit_search("GradientBoosting", gb, gb_param_dist, n_iter=25, cv=3)
et_search = fit_search("ExtraTrees", et, et_param_dist, n_iter=25, cv=3)
ridge_search = fit_search("Ridge", ridge, ridge_param_dist, n_iter=10, cv=3)  # Meno iter per modelli lineari
elastic_search = fit_search("ElasticNet", elastic, elastic_param_dist, n_iter=15, cv=3)

# Raccolta dei risultati
searches = {
    "RandomForest": rf_search,
    "HistGradientBoosting": hgb_search,
    "XGBoost": xgb_search,
    "LightGBM": lgbm_search,
    "GradientBoosting": gb_search,
    "ExtraTrees": et_search,
    "Ridge": ridge_search,
    "ElasticNet": elastic_search,
}

# Confronto finale
print("\n" + "=" * 60)
print("RISULTATI FINALI - CONFRONTO MODELLI")
print("=" * 60)
print(f"{'Modello':<20} {'Best CV MAE':<15} {'Std':<10}")
print("-" * 45)

results = []
for name, search in searches.items():
    cv_scores = cross_val_score(
        search.best_estimator_, X_tr, y_tr, 
        cv=3, scoring="neg_mean_absolute_error"
    )
    mae_mean = -cv_scores.mean()
    mae_std = cv_scores.std()
    results.append((name, mae_mean, mae_std, search.best_estimator_))
    print(f"{name:<20} {mae_mean:<15.4f} {mae_std:<10.4f}")

# Ordina per performance
results.sort(key=lambda x: x[1])
print(f"\n🏆 Miglior modello: {results[0][0]} (MAE: {results[0][1]:.4f} ± {results[0][2]:.4f})")

# Salva il miglior modello
best_model = results[0][3]
print(f"\nModello selezionato salvato come 'best_model'")

TRAINING MODELLI

[*] Tuning RandomForest...


Tuning RandomForest:   0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

[+] Best MAE (cv) RandomForest: 0.7673
[+] Best params RandomForest:
{'model__max_depth': 12,
 'model__max_features': 0.5,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__n_estimators': 400}

[*] Tuning HistGradientBoosting...


Tuning HistGradientBoosting:   0%|          | 0/75 [00:00<?, ?it/s]

...

c:\Users\dswal\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[+] Best MAE (cv) Ridge: 1.4735
[+] Best params Ridge:
{'model__alpha': 0.1}

[*] Tuning ElasticNet...


Tuning ElasticNet:   0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

[+] Best MAE (cv) ElasticNet: 1.6327
[+] Best params ElasticNet:
{'model__alpha': 0.1, 'model__l1_ratio': 0.1}

RISULTATI FINALI - CONFRONTO MODELLI
Modello              Best CV MAE     Std       
---------------------------------------------
RandomForest         0.7673          0.0363    
HistGradientBoosting 0.6020          0.0278    
XGBoost              0.6062          0.0257    
LightGBM             0.6107          0.0273    
GradientBoosting     0.5289          0.0068    
ExtraTrees           0.6121          0.0164    
Ridge                1.4735          0.0209    
ElasticNet           1.6327          0.0665    

🏆 Miglior modello: GradientBoosting (MAE: 0.5289 ± 0.0068)

Modello selezionato salvato come 'best_model'


In [8]:
# -----------------------
# 7) Valutazione su validation (aggiornata per tutti i modelli)
# -----------------------
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score, confusion_matrix
)

def evaluate(model, X_va, y_va, model_name, print_cm=True):
    # Predizione continua
    y_cont = model.predict(X_va).astype(float)
    y_cont = np.clip(y_cont, 0, 10)

    # Versione classificata per classi 0..10
    y_cls = np.rint(y_cont).astype(int)
    y_true_cls = np.rint(y_va).astype(int)

    # Metriche
    mae = mean_absolute_error(y_va, y_cont)
    rmse = np.sqrt(mean_squared_error(y_va, y_cont))
    r2 = r2_score(y_va, y_cont)

    # Alcune metriche “di utilità”
    medae = np.median(np.abs(y_va - y_cont))
    within_05 = np.mean(np.abs(y_va - y_cont) <= 0.5)  # % entro mezzo punto
    within_10 = np.mean(np.abs(y_va - y_cont) <= 1.0)  # % entro un punto
    exact_cls = np.mean(y_true_cls == y_cls)           # % match esatto della classe arrotondata

    print(f"\n=== {model_name} on validation ===")
    print(f"MAE:   {mae:.4f}")
    print(f"RMSE:  {rmse:.4f}")
    print(f"R^2:   {r2:.4f}")
    print(f"MedAE: {medae:.4f}")
    print(f"% entro 0.5: {within_05*100:6.2f}%   % entro 1.0: {within_10*100:6.2f}%   % match classe: {exact_cls*100:6.2f}%")

    # Confusion matrix 0..10 (su target arrotondato)
    cm = confusion_matrix(y_true_cls, y_cls, labels=list(range(11)))
    cm_df = pd.DataFrame(
        cm,
        index=[f"true_{i}" for i in range(11)],
        columns=[f"pred_{i}" for i in range(11)]
    )
    if print_cm:
        print("\nConfusion matrix (rounded 0-10):")
        print(cm_df)

    return {
        "name": model_name,
        "mae": mae,
        "rmse": rmse,
        "r2": r2,
        "medae": medae,
        "within_05": within_05,
        "within_10": within_10,
        "exact_cls": exact_cls,
        "y_cont": y_cont,
        "y_cls": y_cls,
        "cm_df": cm_df
    }

# Valuta tutti i best_estimator_ ottenuti dallo step di tuning
val_results = []
print("\n" + "=" * 60)
print("VALUTAZIONE SU VALIDATION - TUTTI I MODELLI")
print("=" * 60)

for name, search in searches.items():
    res = evaluate(search.best_estimator_, X_va, y_va, name, print_cm=True)
    val_results.append(res)

# Tabella riassuntiva ordinata per MAE
summary_rows = []
for r in val_results:
    summary_rows.append([
        r["name"], r["mae"], r["rmse"], r["r2"],
        r["medae"], r["within_05"], r["within_10"], r["exact_cls"]
    ])

summary_df = pd.DataFrame(
    summary_rows,
    columns=["Model", "MAE", "RMSE", "R2", "MedAE", "Within 0.5", "Within 1.0", "Exact Class"]
).sort_values(by="MAE")

print("\n" + "-" * 60)
print("RIEPILOGO ORDINATO PER MAE (validation)")
print("-" * 60)
print(summary_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

# Migliore su validation per MAE
best_val = min(val_results, key=lambda r: r["mae"])
best_name_val = best_val["name"]
best_model_val = searches[best_name_val].best_estimator_
print(f"\n[***] Best model on validation (MAE): {best_name_val} (MAE={best_val['mae']:.4f})")

# (Opzionale) se vuoi che 'best_model' sia anche quello migliore su validation:
best_model = best_model_val
print("Aggiornato 'best_model' con il migliore su validation.")



VALUTAZIONE SU VALIDATION - TUTTI I MODELLI

=== RandomForest on validation ===
MAE:   0.6366
RMSE:  0.9118
R^2:   0.8789
MedAE: 0.4333
% entro 0.5:  57.18%   % entro 1.0:  78.71%   % match classe:  57.18%

Confusion matrix (rounded 0-10):
         pred_0  pred_1  pred_2  pred_3  pred_4  pred_5  pred_6  pred_7  \
true_0       38      18       2       0       0       0       0       0   
true_1        0      25      27       6       1       0       0       0   
true_2        0       1      23      24      10       0       0       0   
true_3        0       0       7      39       8       0       0       0   
true_4        0       0       1       6      53       1       0       0   
true_5        0       0       0       0       0       0       0       0   
true_6        0       0       0       4      11      17      23       0   
true_7        0       0       0       0       0       0       0       0   
true_8        0       0       0       0       0       3       7      19   
true_9   

In [9]:
# -----------------------
# 8) Importanza delle feature (permutation importance)
# -----------------------
print("\n[*] Calcolo permutation importance sul best model (può essere lento)...")
perm = permutation_importance(best_model, X_va, y_va, scoring="neg_mean_absolute_error", n_repeats=5, random_state=42, n_jobs=-1)
importances = pd.Series(perm.importances_mean, index=X_full.columns).sort_values(ascending=False)
topk = 25
print(f"\nTop {topk} feature (permutation importance):")
print(importances.head(topk))


[*] Calcolo permutation importance sul best model (può essere lento)...

Top 25 feature (permutation importance):
az_zcr               0.260796
ax_kurt              0.211050
ipi_cv               0.204804
ipi_std              0.194618
ay_kurt              0.178284
ax_band_4000_8000    0.144206
az_kurt              0.142413
az_band_4000_8000    0.137975
ay_band_4000_8000    0.114086
ax_min               0.107565
az_band_1000_2000    0.097723
az_iqr               0.091463
ay_band_0_200        0.088185
ax_zcr               0.084932
az_p10               0.083794
az_rms               0.081748
az_std               0.073736
corr_yz              0.068833
ay_zcr               0.068244
ay_iqr               0.064571
az_band_0_200        0.059786
az_max               0.057752
az_median            0.057740
ax_band_500_1000     0.057240
az_skew              0.055371
dtype: float64


In [ ]:
# -----------------------
# 8) Salvataggio del modello migliore
# -----------------------

# Crea una cartella per i modelli se non esiste
os.makedirs('models', exist_ok=True)

# Timestamp per rendere unico il nome del file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Salvataggio con joblib (raccomandato per modelli scikit-learn)
model_filename = f'../models/best_model_{best_name_val}_{timestamp}.joblib'
joblib.dump(best_model, model_filename)
print(f"Modello salvato con joblib in: {model_filename}")

# Opzionale: salvataggio con pickle (alternativa)
'''model_filename_pkl = f'models/best_model_{best_name_val}_{timestamp}.pkl'
with open(model_filename_pkl, 'wb') as f:
    pickle.dump(best_model, f)
print(f"Modello salvato con pickle in: {model_filename_pkl}")
'''
# Salva anche i metadati del modello migliore
metadata = {
    'model_name': best_name_val,
    'mae': best_val['mae'],
    'rmse': best_val['rmse'],
    'r2': best_val['r2'],
    'medae': best_val['medae'],
    'within_05': best_val['within_05'],
    'within_10': best_val['within_10'],
    'exact_cls': best_val['exact_cls'],
    'timestamp': timestamp,
    'best_params': searches[best_name_val].best_params_
}

metadata_filename = f'models/best_model_metadata_{best_name_val}_{timestamp}.json'
import json
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"Metadati salvati in: {metadata_filename}")

# Per caricare il modello in futuro:
"""
# Caricamento con joblib
loaded_model = joblib.load(model_filename)

# Caricamento con pickle
with open(model_filename_pkl, 'rb') as f:
    loaded_model = pickle.load(f)

# Caricamento metadati
with open(metadata_filename, 'r') as f:
    metadata = json.load(f)
"""

print(f"\n[***] Modello migliore '{best_name_val}' salvato con successo!")
print(f"     MAE: {best_val['mae']:.4f}")
print(f"     File: {model_filename}")
print(f"     Metadati: {metadata_filename}")

Modello salvato con joblib in: models/best_model_GradientBoosting_20250820_112625.joblib
Modello salvato con pickle in: models/best_model_GradientBoosting_20250820_112625.pkl
Metadati salvati in: models/best_model_metadata_GradientBoosting_20250820_112625.json

[***] Modello migliore 'GradientBoosting' salvato con successo!
     MAE: 0.3918
     File: models/best_model_GradientBoosting_20250820_112625.joblib
     Metadati: models/best_model_metadata_GradientBoosting_20250820_112625.json


## Testing

In [11]:
test_df = pd.read_pickle('../data/processed/test_data.pkl')

In [12]:
test_full = expand_features(test_df, array_cols=array_cols, scalar_cols=scalar_cols)
test_full

,velocita,torque,ax_mean,ax_std,ax_min,ax_max,ax_median,ax_p10,ax_p90,ax_rms,...,rpm,pulse_count,ipi_mean,ipi_std,ipi_cv,v_times_t,v_sq,t_sq,v_over_rpm,t_over_rpm
0,100,50,0.000470,0.189005,-0.880544,0.411491,0.018732,-0.262377,0.224168,0.189006,...,54.771784,11,1.082280,0.000435,0.000402,5000,10000,2500,1.825758,0.912879
1,100,50,-0.000101,0.152046,-0.676597,0.352317,0.007319,-0.204195,0.188564,0.152046,...,54.771784,11,1.082236,0.000264,0.000244,5000,10000,2500,1.825758,0.912879
2,100,50,0.000135,0.038641,-0.143656,0.108424,0.001737,-0.052475,0.049498,0.038641,...,54.545455,11,1.082222,0.000392,0.000362,5000,10000,2500,1.833333,0.916667
3,100,50,-0.000111,0.188699,-0.864913,0.400202,0.017988,-0.262501,0.223547,0.188699,...,54.771784,11,1.082217,0.000132,0.000122,5000,10000,2500,1.825758,0.912879
4,100,50,-0.000034,0.150834,-0.609979,0.322295,0.007443,-0.202706,0.187075,0.150834,...,54.771784,11,1.082207,0.000194,0.000180,5000,10000,2500,1.825758,0.912879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,3600,100,-0.003586,1.274652,-4.568457,5.180049,-0.024873,-1.646833,1.676321,1.274657,...,2006.557377,102,0.030001,0.000025,0.000837,360000,12960000,10000,1.794118,0.049837
796,3600,100,-0.001434,1.061359,-4.559400,5.011086,-0.003225,-1.355923,1.358069,1.061360,...,1993.548387,103,0.030001,0.000024,0.000804,360000,12960000,10000,1.805825,0.050162
797,3600,100,0.003537,1.069601,-5.260684,5.119634,0.013212,-1.353355,1.342935,1.069607,...,2012.903226,104,0.030001,0.000024,0.000805,360000,12960000,10000,1.788462,0.049679
798,3600,100,-0.007096,1.067926,-5.602705,5.226941,0.007940,-1.363565,1.328544,1.067949,...,2012.903226,104,0.030001,0.000024,0.000803,360000,12960000,10000,1.788462,0.049679


In [ ]:
# Definiamo le classi da 0 a 10
CLASSES = np.arange(11)

def reg_to_probs(y_cont, classes=CLASSES, sigma=0.75):
    """
    Converte una previsione continua (da un modello di regressione) 
    in una distribuzione di probabilità su un insieme discreto di classi.

    Funzionamento:
    1. Clippa i valori previsti (`y_cont`) ai limiti min/max delle classi ammesse.
    2. Calcola la distanza di ogni previsione da ciascuna classe.
    3. Applica una funzione gaussiana centrata sul valore previsto 
       (larghezza controllata da `sigma`) per ottenere un peso a ogni classe.
    4. Normalizza i pesi in modo che sommino a 1, ottenendo una distribuzione di probabilità.

    Parametri:
    - y_cont: array di previsioni continue.
    - classes: array di valori discreti delle classi.
    - sigma: deviazione standard della "campana" gaussiana usata per assegnare i pesi.

    Output:
    - Matrice (n_samples x n_classes) con la probabilità di appartenenza a ciascuna classe.
    """
    y_cont = np.clip(y_cont.astype(float), classes.min(), classes.max())
    d = classes[None, :] - y_cont[:, None]
    w = np.exp(-0.5 * (d / sigma)**2)
    return w / w.sum(axis=1, keepdims=True)

# Predizioni continue
y_cont = best_model.predict(test_full).astype(float)
y_cont = np.clip(y_cont, 0, 10)

# Pseudo-probabilità (NUMERICHE)
probs = reg_to_probs(y_cont)
probs_df_num = pd.DataFrame(probs, columns=[f"prob_{c}" for c in CLASSES])

# ---- Confidenza binaria ----
# 1) soglia sulla max-prob
max_prob = probs_df_num.max(axis=1)

# 2) entropia normalizzata in [0,1]
eps = 1e-12
entropy = -(probs_df_num * np.log(probs_df_num + eps)).sum(axis=1) / np.log(len(CLASSES))

# soglie (tunable)
THRESH_MAX = 0.50     # es. almeno 50% sulla classe più probabile
THRESH_ENT = 0.70     # entropia non troppo alta (<= 0.70)

high_conf = ((max_prob >= THRESH_MAX) & (entropy <= THRESH_ENT)).astype(int)

# (Opzionale) criterio alternativo o aggiuntivo: margine top1-top2
# top2 = np.partition(probs, -2, axis=1)[:, -2]
# margin = max_prob - top2
# high_conf = ((max_prob >= THRESH_MAX) & (entropy <= THRESH_ENT) & (margin >= 0.15)).astype(int)

# ---- Formattazione per output umano ----
probs_df_fmt = (probs_df_num).round(2).astype(str)

# Unisci con gli ID e la confidenza
df_with_probs = pd.concat(
    [test_df[["id"]], probs_df_fmt],
    axis=1
)
df_with_probs["high_confidence"] = high_conf  # 0 = bassa, 1 = alta
df_with_probs = df_with_probs.sort_values(['id']).reset_index(drop=True)
print(df_with_probs)


      id prob_0 prob_1 prob_2 prob_3 prob_4 prob_5 prob_6 prob_7 prob_8  \
0      1    0.0    0.0   0.05   0.37   0.48    0.1    0.0    0.0    0.0   
1      2    0.0   0.02   0.27   0.53   0.18   0.01    0.0    0.0    0.0   
2      3    0.0    0.0    0.0    0.0   0.03    0.3   0.51   0.15   0.01   
3      4    0.0   0.12   0.49   0.35   0.04    0.0    0.0    0.0    0.0   
4      5    0.0    0.0    0.0    0.0    0.0   0.12   0.49   0.35   0.04   
..   ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
795  796    0.0    0.0    0.0    0.0    0.0   0.02   0.26   0.53   0.18   
796  797   0.14   0.51   0.31   0.03    0.0    0.0    0.0    0.0    0.0   
797  798    0.0    0.1   0.47   0.37   0.05    0.0    0.0    0.0    0.0   
798  799    0.0    0.0    0.0   0.01   0.12    0.5   0.34   0.04    0.0   
799  800    0.2   0.54   0.25   0.02    0.0    0.0    0.0    0.0    0.0   

    prob_9 prob_10  high_confidence  
0      0.0     0.0                0  
1      0.0     0.0     

## Altro testing ancora

In [38]:
import numpy as np
from scipy.stats import norm

def normal_cdf(z):
    return norm.cdf(z)

def class_probs_gaussian(mu, sigma, n_classes=11):
    mu = np.asarray(mu, dtype=float).ravel()
    sigma = max(float(sigma), 1e-12)
    edges = np.arange(-0.5, n_classes - 0.5 + 1.0, 1.0)  # n_classes+1 soglie
    z = (edges[None, :] - mu[:, None]) / sigma
    cdf = normal_cdf(z)
    probs = np.diff(cdf, axis=1)
    probs = np.clip(probs, 1e-12, 1.0)
    probs /= probs.sum(axis=1, keepdims=True)
    return probs

def nll_given_sigma(y_true, mu_pred, sigma, n_classes=11):
    mu_pred = np.asarray(mu_pred, dtype=float).ravel()
    y_true = np.asarray(y_true, dtype=int).ravel()
    y_true = np.clip(y_true, 0, n_classes-1)
    sigma = max(float(sigma), 1e-12)
    edges = np.arange(-0.5, n_classes - 0.5 + 1.0, 1.0)
    z_hi = (edges[y_true + 1] - mu_pred) / sigma
    z_lo = (edges[y_true]     - mu_pred) / sigma
    p = np.clip(normal_cdf(z_hi) - normal_cdf(z_lo), 1e-12, 1.0)
    return -np.sum(np.log(p))

def fit_sigma_mle(y_true, mu_pred, n_classes=11):
    s0 = max(1e-3, float(np.sqrt(np.mean((np.asarray(y_true).ravel()
                                          - np.asarray(mu_pred).ravel())**2))))
    grid = s0 * np.logspace(-1.0, 1.0, 31)
    best_sigma, best_nll = None, np.inf
    for s in grid:
        val = nll_given_sigma(y_true, mu_pred, s, n_classes)
        if val < best_nll:
            best_nll, best_sigma = val, s
    for _ in range(2):
        local = np.linspace(best_sigma*0.5, best_sigma*1.5, 21)
        for s in local:
            val = nll_given_sigma(y_true, mu_pred, s, n_classes)
            if val < best_nll:
                best_nll, best_sigma = val, s
    return float(best_sigma)


In [39]:
def rmse_sigma(y_true, mu_pred):
    resid = np.asarray(y_true).ravel() - np.asarray(mu_pred).ravel()
    return float(np.sqrt(np.mean(resid**2)))

In [41]:
# ==== Stima di sigma sulla validation ====
# Predizioni continue del migliore su validation
y_cont_va = best_model.predict(X_va).astype(float)
# stesso clipping che usi nella evaluate (coerente con il tuo workflow)
y_cont_va = np.clip(y_cont_va, 0, 10)

# Opzione semplice: sigma = RMSE su validation
sigma_rmse = rmse_sigma(y_va, y_cont_va)

# Opzione migliore (comunque semplice): MLE 1D
sigma_mle = fit_sigma_mle(np.rint(y_va).astype(int), y_cont_va, n_classes=11)

print(f"\nSigma (RMSE): {sigma_rmse:.4f}")
print(f"Sigma (MLE):  {sigma_mle:.4f}")

# Scegli quale usare
sigma = sigma_mle  # oppure sigma_rmse se preferisci la via più rapida
print(f"Sigma scelto: {sigma:.4f}")



Sigma (RMSE): 0.6141
Sigma (MLE):  0.5530
Sigma scelto: 0.5530


In [ ]:
# Predizioni continue
y_cont = best_model.predict(test_full).astype(float)
y_cont = np.clip(y_cont, 0, 10)

# Probabilità ordinali con sigma scelto (mle o rmse)
probs = class_probs_gaussian(y_cont, sigma, n_classes=11)
probs_df_num = pd.DataFrame(probs, columns=[f"prob_{c}" for c in CLASSES])

# ---- Confidenza binaria ----
max_prob = probs_df_num.max(axis=1)
eps = 1e-12
entropy = -(probs_df_num * np.log(probs_df_num + eps)).sum(axis=1) / np.log(len(CLASSES))

THRESH_MAX = 0.50
THRESH_ENT = 0.70
high_conf = ((max_prob >= THRESH_MAX) & (entropy <= THRESH_ENT)).astype(int)

# (facoltativo) classe MAP e/o margine
yhat_map = probs_df_num.values.argmax(axis=1)
# top2 = np.partition(probs, -2, axis=1)[:, -2]
# margin = max_prob - top2

# ---- Output
probs_df_fmt = probs_df_num.round(2).astype(str)
df_with_probs = pd.concat([test_df[["id"]], probs_df_fmt], axis=1)
df_with_probs["high_confidence"] = high_conf
df_with_probs = df_with_probs.sort_values(["id"]).reset_index(drop=True)
print(df_with_probs.head())


   id prob_0 prob_1 prob_2 prob_3 prob_4 prob_5 prob_6 prob_7 prob_8 prob_9  \
0   1    0.0    0.0   0.02   0.38   0.54   0.06    0.0    0.0    0.0    0.0   
1   2    0.0   0.01   0.24   0.62   0.13    0.0    0.0    0.0    0.0    0.0   
2   3    0.0    0.0    0.0    0.0   0.01   0.28   0.61    0.1    0.0    0.0   
3   4    0.0   0.07   0.56   0.35   0.02    0.0    0.0    0.0    0.0    0.0   
4   5    0.0    0.0    0.0    0.0    0.0   0.07   0.56   0.35   0.02    0.0   

  prob_10  high_confidence  
0     0.0                1  
1     0.0                1  
2     0.0                1  
3     0.0                1  
4     0.0                1  


c:\Users\dswal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


: 

## Altro testing

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import coo_matrix

def local_sigmas(X, k=10, metric='euclidean'):
    # X: (n,d) L2-normalizzato se usi distanza euclidea o coseno.
    nn = NearestNeighbors(n_neighbors=k+1, metric=metric).fit(X)
    dists, _ = nn.kneighbors(X)
    # robusto: mediana delle distanze ai primi k vicini (salta lo 0 a colonna 0)
    return np.median(dists[:, 1:k+1], axis=1)

def gaussian_affinity_knn(X, sigmas, knn_graph_k=15, metric='euclidean'):
    # Grafo sparso simmetrico: w_ij = exp(-d^2 / (σ_i σ_j))
    G = NearestNeighbors(n_neighbors=knn_graph_k+1, metric=metric).fit(X)
    dists, idx = G.kneighbors(X)
    rows = np.repeat(np.arange(len(X)), knn_graph_k)
    cols = idx[:, 1:].ravel()
    d = dists[:, 1:].ravel()
    denom = (sigmas[rows] * sigmas[cols]) + 1e-12
    w = np.exp(-(d**2) / denom)
    W = coo_matrix((w, (rows, cols)), shape=(len(X), len(X))).tocsr()
    # simmetrizza (mutual kNN)
    W = 0.5 * (W + W.T)
    return W

def global_sigma_from_local(sigmas):
    return float(np.median(sigmas))


In [26]:
# Definiamo le classi da 0 a 10
CLASSES = np.arange(11)

def local_sigmas(X, k=10, metric='euclidean'):
    # X: (n,d) L2-normalizzato se usi distanza euclidea o coseno.
    nn = NearestNeighbors(n_neighbors=k+1, metric=metric).fit(X)
    dists, _ = nn.kneighbors(X)
    # robusto: mediana delle distanze ai primi k vicini (salta lo 0 a colonna 0)
    return np.median(dists[:, 1:k+1], axis=1)

'''# NON USATO
def gaussian_affinity_knn(X, sigmas, knn_graph_k=15, metric='euclidean'):
    # Grafo sparso simmetrico: w_ij = exp(-d^2 / (σ_i σ_j))
    G = NearestNeighbors(n_neighbors=knn_graph_k+1, metric=metric).fit(X)
    dists, idx = G.kneighbors(X)
    rows = np.repeat(np.arange(len(X)), knn_graph_k)
    cols = idx[:, 1:].ravel()
    d = dists[:, 1:].ravel()
    denom = (sigmas[rows] * sigmas[cols]) + 1e-12
    w = np.exp(-(d**2) / denom)
    W = coo_matrix((w, (rows, cols)), shape=(len(X), len(X))).tocsr()
    # simmetrizza (mutual kNN)
    W = 0.5 * (W + W.T)
    return W'''

def global_sigma_from_local(sigmas):
    return float(np.median(sigmas))

def reg_to_probs_adaptive(y_cont, X_features, classes=CLASSES, k=10, metric='euclidean'):
    """
    Converte una previsione continua (da un modello di regressione) 
    in una distribuzione di probabilità su un insieme discreto di classi
    utilizzando sigma adattiva basata sulla densità locale dei dati.

    Funzionamento:
    1. Calcola le sigma locali per ogni sample basandosi sui k-nearest neighbors
    2. Clippa i valori previsti (`y_cont`) ai limiti min/max delle classi ammesse.
    3. Calcola la distanza di ogni previsione da ciascuna classe.
    4. Applica una funzione gaussiana con sigma adattiva per ottenere un peso a ogni classe.
    5. Normalizza i pesi in modo che sommino a 1, ottenendo una distribuzione di probabilità.

    Parametri:
    - y_cont: array di previsioni continue.
    - X_features: array delle features utilizzate per calcolare le sigma locali.
    - classes: array di valori discreti delle classi.
    - k: numero di nearest neighbors per calcolare sigma locale.
    - metric: metrica di distanza per i nearest neighbors.

    Output:
    - Matrice (n_samples x n_classes) con la probabilità di appartenenza a ciascuna classe.
    - Array delle sigma locali utilizzate.
    """
    # Calcola le sigma locali basate sui nearest neighbors delle features
    sigmas = local_sigmas(X_features, k=k, metric=metric)

    '''low, high = np.percentile(sigmas_used, [5, 95])
    sigmas = np.clip(sigmas, low, high)'''
    sigmas = np.clip(sigmas, 0.3, 2.0)
    
    # Clippa le previsioni continue
    y_cont = np.clip(y_cont.astype(float), classes.min(), classes.max())
    
    # Calcola le distanze da ogni classe
    d = classes[None, :] - y_cont[:, None]  # (n_samples, n_classes)
    
    # Applica la gaussiana con sigma adattiva per ogni sample
    # Ogni sample usa la propria sigma locale
    w = np.exp(-0.5 * (d / sigmas[:, None])**2)
    
    # Normalizza per ottenere probabilità
    probs = w / w.sum(axis=1, keepdims=True)
    
    return probs, sigmas

# ---- CODICE PRINCIPALE ----

# Predizioni continue
y_cont = best_model.predict(test_full).astype(float)
y_cont = np.clip(y_cont, 0, 10)

# Assumendo che test_full contenga le features normalizzate
# Se hai bisogno di normalizzare, decommentare le righe seguenti:
# from sklearn.preprocessing import normalize
# test_features_norm = normalize(test_full, norm='l2')

# Pseudo-probabilità con sigma adattiva
probs, sigmas_used = reg_to_probs_adaptive(
    y_cont, 
    test_full,  # o test_features_norm se normalizzato
    k=10,  # numero di nearest neighbors per sigma locale
    metric='euclidean'  # o 'cosine' se usi features normalizzate
)

probs_df_num = pd.DataFrame(probs, columns=[f"prob_{c}" for c in CLASSES])

# ---- Confidenza binaria (aggiornata con informazioni sulle sigma) ----
# 1) soglia sulla max-prob
max_prob = probs_df_num.max(axis=1)

# 2) entropia normalizzata in [0,1]
eps = 1e-12
entropy = -(probs_df_num * np.log(probs_df_num + eps)).sum(axis=1) / np.log(len(CLASSES))

# 3) confidenza basata su sigma locale (più bassa sigma = maggiore confidenza)
# Normalizza le sigma in [0,1] per facilità di soglia

sigma_norm = (sigmas_used - sigmas_used.min()) / (sigmas_used.max() - sigmas_used.min() + eps)
sigma_confidence = 1 - sigma_norm  # sigma bassa -> confidenza alta

# soglie (tunable)
THRESH_MAX = 0.50      # es. almeno 50% sulla classe più probabile
THRESH_ENT = 0.70      # entropia non troppo alta (<= 0.70)
THRESH_SIGMA = 0.30    # confidenza da sigma >= 30%

# Criterio di alta confidenza combinato
high_conf = (
    (max_prob >= THRESH_MAX) & 
    (entropy <= THRESH_ENT) & 
    (sigma_confidence >= THRESH_SIGMA)
).astype(int)

# (Opzionale) criterio alternativo: margine top1-top2
# top2 = np.partition(probs, -2, axis=1)[:, -2]
# margin = max_prob - top2
# high_conf = (
#     (max_prob >= THRESH_MAX) & 
#     (entropy <= THRESH_ENT) & 
#     (sigma_confidence >= THRESH_SIGMA) & 
#     (margin >= 0.15)
# ).astype(int)

# ---- Formattazione per output umano ----
probs_df_fmt = (probs_df_num).round(2).astype(str)

# Unisci con gli ID e la confidenza
df_with_probs = pd.concat(
    [test_df[["id"]], probs_df_fmt],
    axis=1
)
df_with_probs["high_confidence"] = high_conf  # 0 = bassa, 1 = alta
df_with_probs["sigma_local"] = sigmas_used.round(3)  # aggiungi sigma locale per debug
df_with_probs["sigma_confidence"] = sigma_confidence.round(3)  # confidenza da sigma
df_with_probs = df_with_probs.sort_values(['id']).reset_index(drop=True)

# Statistiche informative
print(f"Sigma globale (mediana): {global_sigma_from_local(sigmas_used):.3f}")
print(f"Sigma range: [{sigmas_used.min():.3f}, {sigmas_used.max():.3f}]")
print(f"Samples con alta confidenza: {high_conf.sum()}/{len(high_conf)} ({100*high_conf.mean():.1f}%)")
print("\n", df_with_probs.head(10))

Sigma globale (mediana): 2.000
Sigma range: [0.398, 2.000]
Samples con alta confidenza: 20/800 (2.5%)

    id prob_0 prob_1 prob_2 prob_3 prob_4 prob_5 prob_6 prob_7 prob_8 prob_9  \
0   1   0.04   0.08   0.14   0.19    0.2   0.16    0.1   0.05   0.02   0.01   
1   2   0.07   0.13   0.19   0.21   0.18   0.12   0.06   0.03   0.01    0.0   
2   3    0.0   0.01   0.03   0.08   0.13   0.19    0.2   0.17   0.11   0.06   
3   4   0.11   0.17   0.21    0.2   0.15   0.09   0.04   0.01    0.0    0.0   
4   5    0.0   0.01   0.02   0.05    0.1   0.16    0.2   0.19   0.14   0.08   
5   6    0.0    0.0   0.01   0.04   0.09   0.15   0.19    0.2   0.16    0.1   
6   7    0.1   0.16   0.21   0.21   0.16    0.1   0.05   0.02   0.01    0.0   
7   8    0.0   0.01   0.02   0.05    0.1   0.16    0.2   0.19   0.14   0.08   
8   9    0.0   0.01   0.04   0.08   0.14   0.19    0.2   0.16    0.1   0.05   
9  10   0.03   0.07   0.13   0.19    0.2   0.17   0.11   0.06   0.02   0.01   

  prob_10  high_confidence

c:\Users\dswal\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
